In [23]:
from datasets import load_dataset

dataset = load_dataset("ag_news")
train_data = dataset['train']

import pandas as pd
df = pd.DataFrame(train_data)


In [24]:
df_sampled = df.groupby('label').apply(lambda x: x.sample(1000)).reset_index(drop=True)


/tmp/ipykernel_20546/1472204820.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = vectorizer.fit_transform(df_sampled['text'])


In [26]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L12-v2')
X_sbert = model.encode(df_sampled['text'], show_progress_bar=True)


Batches:   0%|          | 0/125 [00:00<?, ?it/s]

In [27]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import plotly.express as px

X_tfidf_pca = PCA(n_components=2).fit_transform(X_tfidf.toarray())
X_sbert_pca = PCA(n_components=2).fit_transform(X_sbert)

X_tfidf_tsne = TSNE(n_components=2, perplexity=30).fit_transform(X_tfidf.toarray())
X_sbert_tsne = TSNE(n_components=2, perplexity=30).fit_transform(X_sbert)

X_tfidf_umap = umap.UMAP(n_components=2).fit_transform(X_tfidf.toarray())
X_sbert_umap = umap.UMAP(n_components=2).fit_transform(X_sbert)

df_sampled["label_name"] = df_sampled["label"].map({
    0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"
})



/home/cezary/DataMining/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/cezary/DataMining/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [28]:
px.scatter(x=X_tfidf_pca[:,0], y=X_tfidf_pca[:,1], color=df_sampled["label_name"], title="TF-IDF PCA").show()
px.scatter(x=X_sbert_pca[:,0], y=X_sbert_pca[:,1], color=df_sampled["label_name"], title="SBERT PCA").show()


In [29]:
px.scatter(x=X_tfidf_tsne[:,0], y=X_tfidf_tsne[:,1], color=df_sampled["label_name"], title="TF-IDF TSNE").show()
px.scatter(x=X_sbert_tsne[:,0], y=X_sbert_tsne[:,1], color=df_sampled["label_name"], title="SBERT TSNE").show()

In [30]:
px.scatter(x=X_tfidf_umap[:,0], y=X_tfidf_umap[:,1], color=df_sampled["label_name"], title="TF-IDF UMAP").show()
px.scatter(x=X_sbert_umap[:,0], y=X_sbert_umap[:,1], color=df_sampled["label_name"], title="SBERT UMAP").show()